In [1]:
# Packages, seed and path
## packages
from edge_sim_py import *
import math
import os
import random
import msgpack
import pandas as pd
import matplotlib.pyplot as plt

## seed
import torch
torch.manual_seed(5)
import random
random.seed(5)
import numpy as np
np.random.seed(5)

## path
algo_name = "mab_lm"

/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Functions
# def custom_collect_method(self) -> dict: # Custom collect method to measure the power consumption of each server
#     metrics = {
#         "Instance ID": self.id,
#         "Power Consumption": self.get_power_consumption(),
#     }
#     return metrics

edge_rewards = {}  #Initialise overall dict for rewards of migrating to a edge server
edge_counts = {} #Inititalise the count dict to count the number of times the service has been migrated to a edge server
edge_ucb_scores = {} #Initialise the upper confidence bound score dict

power_list = list() # List to store total power consumption everytime the task scheduling algorithm is used


def my_algorithm(parameters):

    
    print("\n\n")
    total_power = 0 #We sum the power consumption after migrating each service
    

    for service in Service.all(): #Iterate over every service


        
        
        if not service.being_provisioned: #If service needs to be migrated

            

            for edge_server in EdgeServer.all(): #iterate through every edge server


                if edge_counts[service][edge_server] == 0:
                    # If an edge server has not been explored yet, assign a high UCB score
                    ucb_score = float('inf')
                else:
                    # Calculate the UCB score using the Upper Confidence Bound formula
                    exploration_term = math.sqrt(math.log(sum(edge_counts[service].values())) / edge_counts[service][edge_server])
                    ucb_score = edge_rewards[service][edge_server] / edge_counts[service][edge_server] + 0.5 * exploration_term


                edge_ucb_scores[service][edge_server] = ucb_score
            # selected_edge_server = max(edge_ucb_scores[service], key=edge_ucb_scores[service].get)
            

            #sort every edge server by upper confidence bound score
            edge_servers = sorted(
                EdgeServer.all(),
                key=lambda s: edge_ucb_scores[service][s],
                reverse=True,
            )



            
            for selected_edge_server in edge_servers: #iterate through sorted list

                if selected_edge_server.has_capacity_to_host(service=service):
                    # If server can host our service
                

                    # To save resource, we don't migrate it back to the current edge server
                    if service.server != selected_edge_server:
                        print(f"[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {selected_edge_server}")
                        
                        
                        #Migrate to edge server and increment count
                        service.provision(target_server=selected_edge_server)
                        edge_counts[service][selected_edge_server] += 1
                        
                        #get the sum of powerconsumption of each edge server
                        power = 0
                        for iter_edge_server in EdgeServer.all():
                            power = power + iter_edge_server.get_power_consumption()
                        # After start migrating the service we can move on to the next service
                        total_power += power
                        break

            

        for edge_server in EdgeServer.all():
            #print(edge_server.cpu,edge_server.memory,edge_server.disk)
            edge_rewards[service][edge_server] += 1/edge_server.get_power_consumption()

    #Append to power_list for plotting
    power_list.append(total_power)

def stopping_criterion(model: object):    
    # As EdgeSimPy will halt the simulation whenever this function returns True,
    # its output will be a boolean expression that checks if the current time step is 600
    return model.schedule.steps == 1000

In [3]:
simulator = Simulator(
    dump_interval=5,
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset
#simulator.initialize(input_file="sample_dataset3.json")
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

#Assigning the custom collect method
#EdgeServer.collect = custom_collect_method

#Initialise dicts for each service
for service in Service.all():
    if service not in edge_rewards:
        edge_rewards[service] = {}

    if service not in edge_counts:
        edge_counts[service] = {}

    if service not in edge_ucb_scores:
        edge_ucb_scores[service] = {}


    for edge_server in EdgeServer.all():
        if edge_server not in edge_rewards:
                    # Initialize rewards and counts for each service, and for each service, eachedge server
            edge_rewards[service][edge_server] = 0
            edge_counts[service][edge_server] = 0

# Executing the simulation
simulator.run_model()




[STEP 1] Migrating Service_1 From None to EdgeServer_1
[STEP 1] Migrating Service_2 From None to EdgeServer_1
[STEP 1] Migrating Service_3 From None to EdgeServer_1
[STEP 1] Migrating Service_4 From None to EdgeServer_1
[STEP 1] Migrating Service_5 From None to EdgeServer_1
[STEP 1] Migrating Service_6 From None to EdgeServer_1






[STEP 3] Migrating Service_1 From EdgeServer_1 to EdgeServer_2
[STEP 3] Migrating Service_2 From EdgeServer_1 to EdgeServer_2






[STEP 5] Migrating Service_1 From EdgeServer_2 to EdgeServer_3
[STEP 5] Migrating Service_2 From EdgeServer_2 to EdgeServer_3






[STEP 7] Migrating Service_1 From EdgeServer_3 to EdgeServer_4
[STEP 7] Migrating Service_2 From EdgeServer_3 to EdgeServer_4






[STEP 9] Migrating Service_1 From EdgeServer_4 to EdgeServer_5
[STEP 9] Migrating Service_2 From EdgeServer_4 to EdgeServer_5
[STEP 9] Migrating Service_3 From EdgeServer_1 to EdgeServer_2
[STEP 9] Migrating Service_4 From EdgeServer_1 to EdgeServer_2
[STEP 9] Migr

In [4]:
# Results
## Retrieving logs dataframe for plot
logs_containerregistry = pd.DataFrame(simulator.agent_metrics["ContainerRegistry"])
logs_edgeserver = pd.DataFrame(simulator.agent_metrics["EdgeServer"])
logs_edgeserver['CPU Usage'] = (logs_edgeserver['CPU Demand']*100)/logs_edgeserver['CPU']
logs_edgeserver['RAM Usage'] = (logs_edgeserver['RAM Demand']*100)/logs_edgeserver['RAM']
logs_networkflow = pd.DataFrame(simulator.agent_metrics["NetworkFlow"])
logs_networkswitch = pd.DataFrame(simulator.agent_metrics["NetworkSwitch"])
logs_service = pd.DataFrame(simulator.agent_metrics["Service"])
logs_user = pd.DataFrame(simulator.agent_metrics["User"])

dt_edge = pd.concat([
    logs_edgeserver[['Object', 'Power Consumption']].groupby(by=['Object']).sum().reset_index(),
    logs_edgeserver[['Object', 'CPU Usage', 'RAM Usage']].groupby(by=['Object']).mean().reset_index(drop=True)
    ], axis=1)
dt_edge_final = pd.DataFrame(dt_edge.mean()).reset_index().rename(columns={'index':'parameters', 0:'values'})
display(dt_edge, dt_edge_final)

/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


,Object,Power Consumption,CPU Usage,RAM Usage
0,EdgeServer_1,176384.159000,10.314685,10.314685
1,EdgeServer_2,176384.159000,10.314685,10.314685
2,EdgeServer_3,87878.702125,13.773726,27.547453
3,EdgeServer_4,87671.065300,13.636364,27.272727
4,EdgeServer_5,87074.966667,17.449217,19.923826
5,EdgeServer_6,76510.850000,9.740260,14.610390


,parameters,values
0,Power Consumption,115317.317015
1,CPU Usage,12.538156
2,RAM Usage,18.330628


In [5]:
print("DELAY SUM", (pd.json_normalize(logs_user['Delays']).sum()).sum())
display(pd.DataFrame(pd.json_normalize(logs_user['Delays']).sum()).T)

DELAY SUM 95020.0


,1,2,3,4,5,6
0,15925.0,15775.0,15785.0,15500.0,16025.0,16010.0


In [6]:
print("DELAY MEAN", (pd.json_normalize(logs_user['Delays']).mean()).mean())
display(pd.DataFrame(pd.json_normalize(logs_user['Delays']).mean()).T)

DELAY MEAN 15.91634938059913


,1,2,3,4,5,6
0,15.940941,15.790791,15.896274,15.609265,16.137966,16.12286


In [7]:
(pd.json_normalize(logs_user['Delays']) > pd.json_normalize(logs_user['Delay_slas'])).sum()

1    0
2    0
3    0
4    8
5    8
6    0
dtype: int64